Procesamiento de datos del segundo fomulario. El que hice en Exactas y Fadu con aprox 15 personas en casa uno. 

Los datos originales estan aca:

https://github.com/IoniUTDT/Magia/tree/master/PilotoFacuFadu

El proyecto Magia es un proyecto que busca medir como responde la gente frente a una situacion novedosa y dificil de explicar en terminos fisicos. La idea es hacer un "truco" (el del buzo en una botella de agua) donde no es obvio porque el globo sube o baja, disfrazado de algo "sorprendente" y pedirle a la gente que complete una encuesta sobre que cree que esta pasando. Buscamos ver cuanto inventa la gente y con que sesgo en funcion de su formacion profesional. 

Los datos de la encuesta son levantados automaticamente mediante el sotware sdaps que genera los formularios y despues mediante escaneo los procesa en bloque sin necesidad de pasar los datos a manos a una compu. Del programa se puede exportar un csv con los datos en bruto. Donde a cada entrada le corresponde una encuesta completada. 

In [13]:
%load_ext autoreload
%autoreload 2
import pandas
from IPython.display import display
import sys
sys.path.append("..")
import leerCSV
data = leerCSV.leerCSV ('./Data_Exp2.csv','Codigos.txt')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


En el siguiente bloque cargamos los datos y los procesamos para que sean mas legibles. Para eso hay que interpretar el csv donde viene guardados. El [csv](./DataValidada_PilotoKonex.csv) se genero automaticamente por el spdaps a partir de escanear el [archivo](./questionnaire.pdf), donde como se puede ver, hay secciones, preguntas y opciones multiples. Cada seccion esta indicada en el encabezado por un primer numero, cada preguntas por un segundo numero y cada opcion por un tercero unidos con un _ luego cada fila representa una entrada donde un 1 representa seleccionado en el caso de las opciones multiples o el valor seleccionado en caso de que se tratara de una escala. 

In [17]:
leerCSV.leerCSV ('./Data_Exp2.csv','Codigos.txt')

,questionnaire_id,global_id,Edad,Profesion,Estudios,Justificacion1,Seguridad1,JustificacionSecundaria,RelevanciaMano,Justificacion2,...,Religion,Reparar,Programar,Correr,Acertijos,Astrologia,Deportes,Debatir,Magia,Electronica
0,['31256'],1002,18-24,Mate/Comp/Sist,Secundaria,Calor,5,"[Magnetismo, Presion]",3,No Se,...,No Gusta,Ni,Gusta,Ni,Encanta,Gusta,Gusta,Encanta,Gusta,Gusta
1,['59406'],1002,18-24,Null,Null,Calor,5,[Magnetismo],2,Electricidad,...,Ni,Ni,Ni,Gusta,Gusta,Gusta,No Gusta,Ni,Gusta,Ni
2,['55654'],1002,18-24,Ingenieria,Universidad,Magnetismo,0,"[Calor, Presion, Electricidad]",4,Magnetismo,...,Odio,Encanta,Encanta,Gusta,Gusta,No Gusta,No Gusta,Gusta,No Gusta,Encanta
3,['15766'],1002,25-40,Bio/Qui/Geo/Pale,Universidad,Presion,6,[Otra],1,Presion,...,Odio,Encanta,Gusta,Gusta,Gusta,Ni,Gusta,Gusta,Gusta,Encanta
4,['61935'],1002,18-24,Bio/Qui/Geo/Pale,Null,Magnetismo,3,[Presion],4,Presion,...,No Gusta,Ni,Ni,Gusta,Gusta,Gusta,Gusta,Gusta,Gusta,Ni
5,['48628'],1002,18-24,Bio/Qui/Geo/Pale,Secundaria,Presion,6,[Magnetismo],5,Presion,...,No Gusta,Ni,Ni,No Gusta,Gusta,No Gusta,No Gusta,Gusta,Gusta,No Gusta
6,['41905'],1002,18-24,Bio/Qui/Geo/Pale,Universidad,Magnetismo,3,[Electricidad],4,No Se,...,No Gusta,Gusta,No Gusta,Gusta,Gusta,Ni,Ni,Gusta,Gusta,No Gusta
7,['25751'],1002,18-24,Bio/Qui/Geo/Pale,Universidad,Magnetismo,5,[Presion],2,Magnetismo,...,No Gusta,Gusta,No Gusta,Gusta,Encanta,No Gusta,Encanta,Gusta,Gusta,No Gusta
8,['57557'],1002,18-24,Bio/Qui/Geo/Pale,Universidad,Magnetismo,9,[Electricidad],1,No Se,...,No Gusta,Gusta,Gusta,Encanta,Encanta,Odio,Encanta,Encanta,Encanta,Ni
9,['59238'],1002,25-40,Bio/Qui/Geo/Pale,Universidad_en_curso,Magnetismo,7,[Electricidad],10,Magnetismo,...,Odio,Encanta,No Gusta,Ni,Gusta,Odio,Ni,Gusta,Gusta,Ni


In [9]:
data = pandas.read_csv('./Data_Exp2.csv')
with open('Codigos.txt') as f:
    lineas = f.readlines()

grupos = []
subGrupo = []

for linea in lineas:
    if linea == "\n":
        grupos.append(subGrupo)
        subGrupo = []
    else:
        subGrupo.append(linea.split('\n')[0])
if not subGrupo == []:
    grupos.append(subGrupo)

for grupo in grupos:
    encabezado = grupo[0]
    multiples = False
    if encabezado[-2:] == '_M':
        multiples=True
        encabezado = encabezado[:-2]
    inicio = grupo[1]
    opciones = grupo[2::]
    
    if len(opciones)>0:
        titulos = [inicio+'_'+str(indice) for indice in range(len(opciones))]
    else:
        titulos = [grupo[1]]
    mapeo = {x:y for x,y in zip(titulos,opciones)}
    
    if len(titulos) > 1:
        if not multiples:
            data[encabezado] = data[titulos].idxmax(axis=1)
            data[encabezado].replace(mapeo, inplace=True)
            data.loc[data[titulos].sum(axis=1)<1, encabezado] = 'NS/NC'
            data.loc[data[titulos].sum(axis=1)>1, encabezado] = 'Null'
        else:
            data[encabezado] = [[]] * len(data)
            for titulo in titulos:
                data[encabezado] = [x if y == 0 else x + [mapeo[titulo]] for x,y in zip(data[encabezado],data[titulo])]
    else:
        data[encabezado] = data[titulos]
    data = data.drop(titulos, 1)

In [10]:
# data = pandas.read_csv('./Data_Exp2.csv')
display (data.head())

,questionnaire_id,global_id,Edad,Profesion,Estudios,Justificacion1,Seguridad1,JustificacionSecundaria,RelevanciaMano,Justificacion2,...,Religion,Reparar,Programar,Correr,Acertijos,Astrologia,Deportes,Debatir,Magia,Electronica
0,['31256'],1002,18-24,Mate/Comp/Sist,Secundaria,Calor,5,"[Magnetismo, Presion]",3,No Se,...,No Gusta,Ni,Gusta,Ni,Encanta,Gusta,Gusta,Encanta,Gusta,Gusta
1,['59406'],1002,18-24,Null,Null,Calor,5,[Magnetismo],2,Electricidad,...,Ni,Ni,Ni,Gusta,Gusta,Gusta,No Gusta,Ni,Gusta,Ni
2,['55654'],1002,18-24,Ingenieria,Universidad,Magnetismo,0,"[Calor, Presion, Electricidad]",4,Magnetismo,...,Odio,Encanta,Encanta,Gusta,Gusta,No Gusta,No Gusta,Gusta,No Gusta,Encanta
3,['15766'],1002,25-40,Bio/Qui/Geo/Pale,Universidad,Presion,6,[Otra],1,Presion,...,Odio,Encanta,Gusta,Gusta,Gusta,Ni,Gusta,Gusta,Gusta,Encanta
4,['61935'],1002,18-24,Bio/Qui/Geo/Pale,Null,Magnetismo,3,[Presion],4,Presion,...,No Gusta,Ni,Ni,Gusta,Gusta,Gusta,Gusta,Gusta,Gusta,Ni
